In [18]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
dt = 1
sigma_a = 0.2 # m/s2
sigma_m = 3 # m


In [38]:
# State transition matrix
F = np.array([[1,0,dt,0,dt**2/2,0],
             [0,1,0,dt,0,dt**2/2],
             [0,0,1,0,dt,0],
             [0,0,0,1,0,dt],
             [0,0,0,0,1,0],
             [0,0,0,0,0,1]], dtype= float)



In [39]:
# Process Noise Matrix
Q = np.array([[dt**4/4, 0,dt**3/2,0,dt**2/2,0],
             [0,dt**4/4, 0,dt**3/2,0,dt**2/2],
             [dt**3/2, 0, dt**2, 0,dt,0],
             [0, dt**3/2,0,dt**2,0,dt],
             [dt**2/2,0,dt,0,1,0],
             [0,dt**2/2,0,dt,0,1]])*sigma_a**2


In [40]:
# measurement uncertainity
R = np.eye(2)*sigma_m**2

In [41]:
measurements = np.array([[301.5,298.23,297.83,300.42,301.94,299.5,305.98,301.25,299.73,299.2],
                        [-401.45,-375.44,-346.15,-320.2,-300.08,-274.12,-253.45,-226.4,-200.65,-171.62]]) 

In [52]:
H = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0]])

In [42]:
# Initialization
init_cov = 500
x = np.array([0,0,0,0,0,0]).reshape(6,1)

P = np.eye(6)*init_cov


In [43]:
P

array([[500.,   0.,   0.,   0.,   0.,   0.],
       [  0., 500.,   0.,   0.,   0.,   0.],
       [  0.,   0., 500.,   0.,   0.,   0.],
       [  0.,   0.,   0., 500.,   0.,   0.],
       [  0.,   0.,   0.,   0., 500.,   0.],
       [  0.,   0.,   0.,   0.,   0., 500.]])

In [56]:
# State Prediction
x_pred = F.dot(x)

# Covariance Prediction
P_pred = F.dot(P).dot(F.T) + Q

In [68]:
## Measure
z = measurements[:,0].reshape(2,1)

## Update
# Kalman Gain
KG = (P_pred.dot(H.T)).dot(np.linalg.inv(H.dot(P_pred).dot(H.T)+R))

# New State Estimate 
x_est = x_pred + np.dot(KG, (z-H.dot(x_pred)))

# New Covariance Estimate
P_est = (np.eye(6)-KG.dot(H)).dot(P_pred)



In [64]:
z

array([[ 301.5 ],
       [-401.45]])

In [65]:
KG

array([[0.99206356, 0.        ],
       [0.        , 0.99206356],
       [0.66138747, 0.        ],
       [0.        , 0.66138747],
       [0.22047425, 0.        ],
       [0.        , 0.22047425]])

In [66]:
x_pred

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [67]:
x_est

array([[ 299.10716396],
       [-398.26391698],
       [ 199.40832091],
       [-265.51399811],
       [  66.47298525],
       [ -88.50938616]])

In [69]:
P_pred

array([[1125.01,    0.  ,  750.02,    0.  ,  250.02,    0.  ],
       [   0.  , 1125.01,    0.  ,  750.02,    0.  ,  250.02],
       [ 750.02,    0.  , 1000.04,    0.  ,  500.04,    0.  ],
       [   0.  ,  750.02,    0.  , 1000.04,    0.  ,  500.04],
       [ 250.02,    0.  ,  500.04,    0.  ,  500.04,    0.  ],
       [   0.  ,  250.02,    0.  ,  500.04,    0.  ,  500.04]])

In [70]:
P_est

array([[  8.92857206,   0.        ,   5.95248719,   0.        ,
          1.98426822,   0.        ],
       [  0.        ,   8.92857206,   0.        ,   5.95248719,
          0.        ,   1.98426822],
       [  5.95248719,   0.        , 503.98617296,   0.        ,
        334.67990582,   0.        ],
       [  0.        ,   5.95248719,   0.        , 503.98617296,
          0.        , 334.67990582],
       [  1.98426822,   0.        , 334.67990582,   0.        ,
        444.91702895,   0.        ],
       [  0.        ,   1.98426822,   0.        , 334.67990582,
          0.        , 444.91702895]])